# Tanzania Wells Analysis
## by Allison Ward, Sarah Prusaitis, and Monica Pecha

Business problem: Unicef is looking to asses the condition of hand pump and communal standpipe wells in the Lake Victoria basin. They have requested that we help them predict the likelihood of a well being functional, in need of repair, or nonfunctional.

The dataset was taken from [Taarifa](https://taarifa.org/) and the [Tanzanian Ministry of Water](https://www.maji.go.tz/). 

# Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading in our datasets to explore
df1 = pd.read_csv('data/test_set_values.csv')
df2= pd.read_csv('data/training_set_labels.csv')
df3 = pd.read_csv('data/training_set_values.csv')

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
df3.head()

In [3]:
# merging dataframes 2 and 3 on id
df4 = df3.merge(df2, on = "id")
df4.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [ ]:
df4.info()

In [4]:
# dropping irrelevant columns
to_drop = ["scheme_name", "id", "public_meeting", "payment_type", "quantity_group", "waterpoint_type", "amount_tsh", "region_code", "district_code", "num_private", "date_recorded", "recorded_by"]
df4.drop(to_drop, axis=1, inplace = True)

In [5]:
# dropping any wells that are not located in the Lake Victoria basin (our stakeholder wants to focus on one region)
df4.drop(df4.index[df4['basin'] != 'Lake Victoria'], inplace = True)
df4.head()

,funder,gps_height,installer,longitude,latitude,wpt_name,basin,subvillage,region,lga,...,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type_group,status_group
1,Grumeti,1399,GRUMETI,34.698766,-2.147466e+00,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,...,user-group,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional
4,Action In A,0,Artisan,31.130847,-1.825359e+00,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,...,other,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,functional
9,Isingiro Ho,0,Artisan,30.626991,-1.257051e+00,Kwapeto,Lake Victoria,Mkonomre,Kagera,Karagwe,...,user-group,never pay,soft,good,enough,shallow well,shallow well,groundwater,hand pump,functional
18,Hesawa,1162,DWE,32.920154,-1.947868e+00,Ngomee,Lake Victoria,Center,Mwanza,Ukerewe,...,user-group,never pay,milky,milky,insufficient,spring,spring,groundwater,other,functional needs repair
21,Dwsp,0,DWE,0.000000,-2.000000e-08,Muungano,Lake Victoria,Ibabachegu,Shinyanga,Bariadi,...,user-group,unknown,unknown,unknown,unknown,shallow well,shallow well,groundwater,hand pump,functional


In [6]:
# dropping any wells that are not a communal standpipe or hand pump
filtered_df = df4[df4['waterpoint_type_group'].isin(['communal standpipe', 'hand pump'])]

In [7]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8330 entries, 1 to 59385
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   funder                 8214 non-null   object 
 1   gps_height             8330 non-null   int64  
 2   installer              8215 non-null   object 
 3   longitude              8330 non-null   float64
 4   latitude               8330 non-null   float64
 5   wpt_name               8330 non-null   object 
 6   basin                  8330 non-null   object 
 7   subvillage             8326 non-null   object 
 8   region                 8330 non-null   object 
 9   lga                    8330 non-null   object 
 10  ward                   8330 non-null   object 
 11  population             8330 non-null   int64  
 12  scheme_management      7515 non-null   object 
 13  permit                 7987 non-null   object 
 14  construction_year      8330 non-null   int64  
 15  extracti

In [8]:
filtered_df['status_group'].value_counts()

status_group
functional                 4527
non functional             2967
functional needs repair     836
Name: count, dtype: int64